# Graph PaySim
This notebook is an example of using Neo4j with Vertex AI.  It takes PaySim data from a Neo4j database, puts that into Feature Store and then runs two classifications on that.  One classification uses the standard PaySim features.  A second uses features engineered using Neo4j Graph Data Science.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/benofben/vertex-ai-samples/blob/master/notebooks/community/neo4j/graph_paysim.ipynb" target="_parent">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/benofben/vertex-ai-samples/tree/master/notebooks/community/neo4j/graph_paysim.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Data Set
The notebook uses a version of the PaySim dataset that has been modified to work with Neo4j's graph database.  PaySim is a synthetic fraud dataset.  The goal is to identify whether or not a given transaction constitutes fraud.  The dataset is [here](https://github.com/voutilad/PaySim).

## Prerequisites
We assume that you've already loaded the PaySim data into a Neo4j instance and have the credentials to connect to that.  You'll also need to install the Neo4j Python driver by running the cell below.

In [1]:
!pip install neo4j

     |████████████████████████████████| 76 kB 2.0 MB/s 
  Created wheel for neo4j: filename=neo4j-4.3.7-py3-none-any.whl size=100642 sha256=ee9bf399a3fcf16cdaeae9ee6ce9975532154689b78ed7e534ca9fe8f9d36c4a
  Stored in directory: /root/.cache/pip/wheels/b5/24/bb/cece9fcfdd5e1aa0683e2533945e1e3f27f70f342ff7e28993
Successfully built neo4j


## Working with Neo4j
In this section we're going to connect to Neo4j and look around the database.  We're going to generate some new features in the dataset using Neo4j's Graph Data Science library.  Finally, we'll load the data into a Pandas dataframe so that it's all ready to put into GCP Feature Store.

In [2]:
import pandas as pd
from neo4j import GraphDatabase

In [4]:
DB_ULR = 'neo4j+s://6c443062.databases.neo4j.io:7687'
DB_USER = 'neo4j'
DB_PASS = 'some password'
DB_NAME = 'neo4j'

In [5]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS))

In [6]:
# Let's have a closer look at transactions
with driver.session(database = DB_NAME) as session:
    result = session.read_transaction( lambda tx: 
        tx.run(
        """
        MATCH (t:Transaction)
        WITH sum(t.amount) AS globalSum, count(t) AS globalCnt
        WITH *, 10^3 AS scaleFactor
        UNWIND ['CashIn', 'CashOut', 'Payment', 'Debit', 'Transfer'] AS txType
        CALL apoc.cypher.run('MATCH (t:' + txType + ')
            RETURN sum(t.amount) as txAmount, count(t) AS txCnt', {})
        YIELD value
        RETURN txType,value.txAmount AS TotalMarketValue,
        100*round(scaleFactor*(toFloat(value.txAmount)/toFloat(globalSum)))
            /scaleFactor AS `%MarketValue`,
        100*round(scaleFactor*(toFloat(value.txCnt)/toFloat(globalCnt)))
            /scaleFactor AS `%MarketTransactions`,
        toInteger(toFloat(value.txAmount)/toFloat(value.txCnt)) AS AvgTransactionValue,
        value.txCnt AS NumberOfTransactions
        ORDER BY `%MarketTransactions` DESC
        """
        ).data()
    )
    df = pd.DataFrame(result)
    display(df)

,txType,TotalMarketValue,%MarketValue,%MarketTransactions,AvgTransactionValue,NumberOfTransactions
0,CashIn,1.040582e+11,40.7,40.5,139347,746751
1,Payment,9.646814e+10,37.8,29.4,177840,542443
2,CashOut,5.385410e+10,21.1,23.0,126842,424574
3,Debit,1.016829e+09,0.4,7.1,7804,130284
4,Transfer,0.000000e+00,0.0,0.0,0,0


## Loading Data into GCP Feature Store
In this section, we'll take our dataframe with newly engineered features and load that into GCP feature store.

## Classification with Vertex AI
In this section, we're going to run two classifiers and compare results.  The first will use the standard PaySim features.  The second will use our new graph features.